# 06 — Full Stack Test (Root-aware)
Auto-detect the **project root** (even when this notebook runs from `/notebooks`) and pick `data/prices_1d` if present, otherwise `data/prices_1h`.
Works with Windows paths containing spaces or apostrophes.


In [ ]:
from pathlib import Path
import sys

# Optional manual override (set to a raw string path or leave as None)
PROJECT_OVERRIDE = None  # e.g. r"C:\\Users\\speed\\Desktop\\Forex CFD's system"

def detect_project_root():
    if PROJECT_OVERRIDE:
        return Path(PROJECT_OVERRIDE)
    here = Path.cwd()
    for p in [here] + list(here.parents):
        if (p/"config"/"baseline.yaml").exists() and ((p/"data"/"prices_1d").exists() or (p/"data"/"prices_1h").exists()):
            return p
    # fallback to CWD
    return here

ROOT = detect_project_root()
DATA_1D = ROOT/"data"/"prices_1d"
DATA_1H = ROOT/"data"/"prices_1h"
CFG = ROOT/"config"/"baseline.yaml"
DATA_DIR = DATA_1D if DATA_1D.exists() else DATA_1H
print("Detected project root:", ROOT)
print("Using data dir:", DATA_DIR)
assert DATA_DIR.exists(), f"Expected data under {ROOT/'data'} (prices_1d or prices_1h)"
assert CFG.exists(), f"Missing baseline config at {CFG}"


## 1) Backtest with guards + vol-carry

In [ ]:
import subprocess, pandas as pd
cmd = [sys.executable, '-m', 'src.exec.backtest_pnl_demo',
       '--cfg', str(CFG), '--folder', str(DATA_DIR), '--costs_csv', str(ROOT/'data'/'costs_per_symbol.csv'),
       '--target_ann_vol','0.12','--vol_lookback','20','--max_leverage','3.0',
       '--mtd_soft','-0.06','--mtd_hard','-0.10',
       '--w_tsmom','1.0','--w_xsec','0.8','--w_mr','0.6','--w_volcarry','0.4',
       '--volcarry_top_q','0.35','--volcarry_bot_q','0.35','--volcarry_lookback','63',
       '--gap_atr_k','3.0','--atr_lookback','14','--vol_spike_mult','3.0','--vol_spike_window','60']
print('RUN:', ' '.join(cmd))
subprocess.check_call(cmd)
pd.read_csv(ROOT/'data'/'pnl_demo_equity.csv').head()


## 2) Small robustness sweep (fast)

In [ ]:
import subprocess
OUT = ROOT/'runs'/'robustness_quickdemo'
OUT.parent.mkdir(parents=True, exist_ok=True)
cmd = [sys.executable, '-m', 'src.exec.sweep_robustness',
       '--cfg', str(CFG), '--folder', str(DATA_DIR), '--costs_csv', str(ROOT/'data'/'costs_per_symbol.csv'),
       '--w_volcarry_grid','0.0,0.3', '--volcarry_topq_grid','0.25,0.45', '--volcarry_botq_grid','0.25,0.45', '--volcarry_lookback_grid','42,84',
       '--target_vol_grid','0.10,0.15','--vol_lookback_grid','15,30','--max_lev_grid','2.0,3.0', '--limit','16',
       '--out_root', str(OUT)]
print('RUN:', ' '.join(cmd))
subprocess.check_call(cmd)
import pandas as pd
pd.read_csv(OUT/'summary.csv').head()


## 3) Build report with Pareto table

In [ ]:
import subprocess
out_html = ROOT/'reports'/'report_quickdemo.html'
out_html.parent.mkdir(parents=True, exist_ok=True)
cmd = [sys.executable, '-m', 'src.exec.make_report_html',
       '--equity_csv', str(ROOT/'data'/'pnl_demo_equity.csv'),
       '--attrib_csv', str(ROOT/'data'/'pnl_demo_attrib_sleeve.csv'),
       '--monthly_csv', str(ROOT/'reports'/'monthly_summary.csv'),
       '--robustness_summary_csv', str(ROOT/'runs'/'robustness_quickdemo'/'summary.csv'),
       '--pareto_top_n','8','--out_html', str(out_html)]
print('RUN:', ' '.join(cmd))
subprocess.check_call(cmd)
print('Report at', out_html)
